In [1]:
from mqt.bench import get_benchmark
from mqt.bench import CompilerSettings, QiskitSettings, TKETSettings, get_benchmark
from qiskit import *
import numpy as np

In [2]:
min_qubit = 3 # maximum number of qubit
max_qubit = 5 # minimum number of qubit
gaps = 1

In [3]:
compiler_settings = CompilerSettings(qiskit=QiskitSettings(optimization_level=1))
circuits= []
for i in range(min_qubit, max_qubit, gaps):
    qc = get_benchmark(benchmark_name="dj",
                       level="nativegates",
                       circuit_size=i,
                       compiler="qiskit",
                       compiler_settings=compiler_settings,
                       provider_name="ionq",)
    circuits.append((i,qc))
    print("---------------ae with qubit ",i,"---------------")
    #print(qc.draw())
print("=================================================================================================")
print(circuits)

---------------ae with qubit  3 ---------------
---------------ae with qubit  4 ---------------
[(3, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7bf126127830>), (4, <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7bf124ba6ae0>)]


In [4]:
from qiskit_aer import AerSimulator
from qiskit import *
from qiskit.providers.fake_provider import *
import numpy as np 
import os
from qiskit.primitives import StatevectorSampler
sampler = AerSimulator()
jobs=[]
for qubit_number,circuit_object in circuits: 
    job = sampler.run([circuit_object], shots=1000)
    results = job.result()
    counts= results.get_counts()
    #print(f" > Counts: {results[0].data["meas"].get_counts()}")
    jobs.append((qubit_number , counts))
print(jobs)

[(3, {'11': 1000}), (4, {'111': 1000})]


In [5]:
def hellinger_fidelity_with_expected(p, q):
    """Compute Hellinger fidelity between measured distribution p (counts)
        and expected distribution q (counts).
        Both p and q should be dictionaries: {bitstring: counts}."""
    p_sum, q_sum = sum(p.values()), sum(q.values())  # Normalize to probability distributions
    if q_sum == 0:
        raise ValueError("Expected distribution invalid (all counts = 0)")
    p_norm = {k: v / p_sum for k, v in p.items()}
    q_norm = {k: v / q_sum for k, v in q.items()}
    overlap = 0.0
    for k in set(p_norm) | set(q_norm):  # union of keys
        overlap += np.sqrt(p_norm.get(k, 0) * q_norm.get(k, 0))  # Compute fidelity    
    return overlap**2

def rescale_fidelity(f, floor, new_floor):
    r = (1-new_floor)/(1-floor) * (f - 1) + 1
    return max(0, min(1, r))

def polarization_fidelity(counts, correct_dist, thermal_dist=None): 
    '''     counts: the measurement outcomes after `num_shots` algorithm runs
                        correct_dist: the distribution we expect to get for the algorithm running perfectly
                        thermal_dist: optional distribution to pass in distribution from a uniform superposition over all states. If `None`: generated as 
                        uniform_dist` with the same qubits as in `counts`'''
    n = len(next(iter(correct_dist)))                            # get length of random key in correct_dist to find how many qubits measured
    counts = {k.zfill(n): v for k, v in counts.items()}          # ensure that all keys in counts are zero padded to this length
    hf = hellinger_fidelity_with_expected(counts, correct_dist)   # calculate hellinger fidelity between measured expectation values and correct distribution
    if n > 16: return {'fidelity': hf, 'hf_fidelity': hf}         # to limit cpu and memory utilization, skip noise correction if more than 16 measured qubits  & if not provided, generate thermal dist based on number of qubits
    if thermal_dist is None: thermal_dist = uniform_dist(n)      # set our fidelity rescaling value as the hellinger fidelity for a depolarized state
    floor = hellinger_fidelity_with_expected(thermal_dist, correct_dist)   # rescale fidelity result so uniform superposition (random guessing) returns fidelity
      
    # rescaled to 0 to provide a better measure of success of the algorithm (polarization)return {'fidelity': rescale_fidelity(hf, floor, 0), 'hf_fidelity': hf}


In [6]:
from fidelity import *
h_f = []
for qubit_number,count in jobs:

    ideal_dist = {}
    if True:  # balanced case
        non_zero_states = [format(i, '0{}b'.format(qubit_number)) for i in range(1, 2**qubit_number)]
        prob = 1 / len(non_zero_states)
        ideal_dist = {state: prob for state in non_zero_states}
    else:  # constant case
        ideal_dist = { '0'*qubit_number: 1.0 }
    hf=hellinger_fidelity_with_expected(counts,ideal_dist)
    h_f.append((qubit_number , hf))
print(h_f)

[(3, 0.14285714285714293), (4, 0.0)]
